In [12]:
from __future__ import absolute_import
from __future__ import print_function
from __future__ import division

import torch as t
from torch.autograd import Variable as V
import torch.nn as nn
import torch.optim as optim

t.manual_seed(777)    # reproducibility

#### 1. input data

In [19]:
#            0,   1,   2,   3,   4
idx2char = ['h', 'i', 'e', 'l', 'o']

# Teach hihell -> ihello
x_data = [[0, 1, 0, 2, 3, 3]]  # hihell
x_one_hot = [[1, 0, 0, 0, 0],  # h 0
                  [0, 1, 0, 0, 0],  # i 1
                  [1, 0, 0, 0, 0],  # h 0
                  [0, 0, 1, 0, 0],  # e 2
                  [0, 0, 0, 1, 0],  # l 3
                  [0, 0, 0, 1, 0]]  # l 3
y_data = [1, 0, 2, 3, 3, 4]    # ihello

# As we have one batch os samples, we will change them to variables only once
inputs = V(t.Tensor(x_one_hot))
labels = V(t.LongTensor(y_data))

if t.cuda.is_available():
    inputs = inputs.cuda()
    labels = labels.cuda()

print("inputs size: ", inputs.size())
print("labels size: ", labels.size())

inputs size:  torch.Size([6, 5])
labels size:  torch.Size([6])


#### 3. define model

In [20]:
# parameters setting
num_classes = 5   
input_size = 5      # one_hot size
hidden_size = 5     # output from RNN(hidden_size=5) to directly predict one-hot
batch_size = 1      # one sequence
sequence_length = 6 # |ihello| = 6
num_layers = 1      # one-layer rnn

In [23]:
# define model
class RNN(nn.Module):
    
    def __init__(self, num_classes, input_size, hidden_size, num_layers):
        super(RNN, self).__init__()
        
        self.num_classes = num_classes
        self.num_layers = num_layers
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.sequence_length = sequence_length

        self.rnn = nn.RNN(input_size=5, hidden_size=5, batch_first=True)
    
    
    def forward(self, x):
        # Initialize hidden and states
        # (batch, num_layers * num_directions, hidden_size) for batch_first=True
        h_0 = V(t.zeros(1, self.num_layers, self.hidden_size))
        if t.cuda.is_available():
            h_0 = h_0.cuda()
        
        # Reshape input
        x = x.view(1, self.sequence_length, self.input_size)
        
        # Propagate input through RNN
        # Input: (batch, seq_len, input_size)(1, 6, 5)
        # h_0: (batch, num_layers * num_directions, hidden_size)
        
        # out: (1, 6, 5)
        out, _ = self.rnn(x, h_0)
        return out.view(-1, num_classes)

In [26]:
# Instantiate RNN model
rnn = RNN(num_classes, input_size, hidden_size, num_layers)
print(rnn)
if t.cuda.is_available():
    rnn.cuda()

# Set loss and optimizer function
# CrossEntropyLoss = LogSoftmax + NLLLoss
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(rnn.parameters(), lr=0.01)

# Train the model
for epoch in range(100):
    outputs = rnn(inputs)
    optimizer.zero_grad()
    
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()
    
    _, idx = outputs.max(1)
    result_str = [idx2char[c] for c in idx.squeeze().cpu().data.numpy()]
    print("epoch: %d, loss: %1.3f" % (epoch+1, loss.data[0]))
    print("Predicted string: ", ''.join(result_str))

print("Learning finished!")

RNN(
  (rnn): RNN(5, 5, batch_first=True)
)
epoch: 1, loss: 1.710
Predicted string:  lieiee
epoch: 2, loss: 1.677
Predicted string:  lieiee
epoch: 3, loss: 1.644
Predicted string:  lieiel
epoch: 4, loss: 1.612
Predicted string:  lieiel
epoch: 5, loss: 1.581
Predicted string:  lielel
epoch: 6, loss: 1.552
Predicted string:  llelel
epoch: 7, loss: 1.524
Predicted string:  llelll
epoch: 8, loss: 1.497
Predicted string:  llelll
epoch: 9, loss: 1.472
Predicted string:  llelll
epoch: 10, loss: 1.447
Predicted string:  llelll
epoch: 11, loss: 1.424
Predicted string:  llelll
epoch: 12, loss: 1.401
Predicted string:  llelll
epoch: 13, loss: 1.378
Predicted string:  llelll
epoch: 14, loss: 1.356
Predicted string:  llelll
epoch: 15, loss: 1.334
Predicted string:  llelll
epoch: 16, loss: 1.312
Predicted string:  llelll
epoch: 17, loss: 1.290
Predicted string:  llelll
epoch: 18, loss: 1.269
Predicted string:  llelll
epoch: 19, loss: 1.248
Predicted string:  llelll
epoch: 20, loss: 1.226
Predicted s